In [1]:
import pandas as pd
import numpy as np

dir = '../data/'

# 1. user_spec Table  
user_spec table은 대출을 신청한 유저의 신용정보에 대한 데이터이다.

In [2]:
user_spec = pd.read_parquet(dir + 'user_spec.parquet')
user_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394216 entries, 0 to 1394215
Data columns (total 17 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   application_id                       1394216 non-null  int32         
 1   user_id                              1394216 non-null  int32         
 2   birth_year                           1381255 non-null  float32       
 3   gender                               1381255 non-null  float32       
 4   insert_time                          1394216 non-null  datetime64[ns]
 5   credit_score                         1289101 non-null  float32       
 6   yearly_income                        1394126 non-null  float32       
 7   income_type                          1394131 non-null  category      
 8   company_enter_month                  1222456 non-null  float32       
 9   employment_type                      1394131 non-null  ca

In [3]:
print(user_spec.shape)
user_spec.head()

(1394216, 17)


,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,1249046,118218,1985.0,1.0,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,20151100.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0
1,954900,553686,1968.0,1.0,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,20070200.0,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1.0,27000000.0
2,137274,59516,1997.0,1.0,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,20210900.0,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5.0,15000000.0
3,1570936,167320,1989.0,1.0,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,20170100.0,정규직,자가,2000000.0,생활비,0.0,NaN,7.0,344000000.0
4,967833,33400,2000.0,1.0,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,20210900.0,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1.0,16000000.0


In [4]:
user_spec.tail()

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
1394211,1864587,489900,2000.0,1.0,2022-03-22 14:55:32,590.0,25000000.0,FREELANCER,202106.0,기타,기타가족소유,5000000.0,사업자금,NaN,NaN,NaN,NaN
1394212,1327066,151422,1955.0,1.0,2022-03-22 01:19:24,980.0,20000000.0,OTHERINCOME,NaN,기타,자가,50000000.0,생활비,NaN,NaN,1.0,NaN
1394213,1319606,173524,1983.0,1.0,2022-03-22 07:34:32,750.0,75000000.0,EARNEDINCOME,200908.0,정규직,자가,100000000.0,대환대출,NaN,NaN,8.0,200000000.0
1394214,1482466,766546,1975.0,1.0,2022-03-22 22:12:35,640.0,50000000.0,EARNEDINCOME,201705.0,정규직,자가,10000000.0,대환대출,NaN,NaN,10.0,117000000.0
1394215,816537,3864,1977.0,0.0,2022-03-22 08:55:14,NaN,35000000.0,FREELANCER,201103.0,기타,자가,20000000.0,생활비,NaN,NaN,NaN,NaN


* 피처들의 종류는 명목형, 수치형 등으로 다양하다.
* NaN 값이 있는 피처들이 확인된다. (조금 뒤에 확인)
* categorical variables는 모두 순서가 없는 nominal이다.

### 데이터 중복 확인

In [7]:
user_spec.drop_duplicates()
user_spec.shape

(1394216, 17)

* 중복된 데이터는 없음

### 데이터 타입 확인  
데이터 타입을 확인하며 데이터 타입별로 관리가 쉬운 메타데이터를 구성  
* role : input, id, target (target은 user_spec 데이터에 존재하지 않음)
* level : nominal, interval, ordinal, binary
* keep : True or False
* dtype : int, float, str

In [8]:
data = []
for f in user_spec.columns:
    # role 설정, 여기서는 target 피처가 없으므로 제외
    if f[-2:] == 'id':
        role = 'id'
    else:
        role = 'input'
    
    # level 설정
    if user_spec[f].dtype == np.float32 or f == 'insert_time':
        if 'rehabilitation' in f or f == 'gender': # 개인회생자 여부 및 납입 완료 여부 또는 성별 피처
            level = 'binary'
        else:
            level = 'interval' # 그 외에 float와 시간 데이터
    else: # category형 변수 및 id
        level = 'nominal'

    # Keep 설정 (keep할지 버릴지에 대한 내용)
    keep = True
    if f[-2:] == 'id':
        keep = False
    
    # dtype 설정
    dtype = user_spec[f].dtype

    f_dict = {
        'varname': f,
        'role' : role,
        'level' : level,
        'keep' : keep,
        'dtype' : dtype
    }

    data.append(f_dict)

meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta = meta.set_index('varname')

In [31]:
meta

,role,level,keep,dtype
varname,,,,
application_id,id,nominal,False,int32
user_id,id,nominal,False,int32
birth_year,input,interval,True,float32
gender,input,binary,True,float32
insert_time,input,interval,True,datetime64[ns]
credit_score,input,interval,True,float32
yearly_income,input,interval,True,float32
income_type,input,nominal,True,category
company_enter_month,input,interval,True,float32


* 메타데이터 사용 예시

In [9]:
meta[(meta.level == 'nominal') & (meta.keep)].index

Index(['income_type', 'employment_type', 'houseown_type', 'purpose'], dtype='object', name='varname')

각 변수들의 role과 level별 숫자 확인

In [10]:
pd.DataFrame({'count': meta.groupby(['role', 'level'])['role'].size()}).reset_index()

,role,level,count
0,id,nominal,2
1,input,binary,3
2,input,interval,8
3,input,nominal,4


* id는 nominal 변수 2개로 구성되어 있다.
* input은 binary 변수 3개, interval 변수 8개, nominal 변수 4개로 구성되어있다.

### Interval variables

In [11]:
pd.options.display.float_format = "{:.4f}".format

v = meta[(meta.level == 'interval') & (meta.keep)].index
user_spec[v].describe()

,birth_year,credit_score,yearly_income,company_enter_month,desired_amount,existing_loan_cnt,existing_loan_amt
count,1381255.0000,1289101.0000,1394126.0000,1222456.0000,1394131.0000,1195660.0000,1080442.0000
mean,1983.2855,638.4448,43453268.0000,5121685.0000,30407270.0000,4.7437,70745904.0000
std,10.6737,124.9724,120840120.0000,8608381.0000,185168720.0000,4.3277,90505600.0000
min,1927.0000,60.0000,0.0000,191109.0000,0.0000,1.0000,0.0000
25%,1976.0000,570.0000,25000000.0000,201902.0000,5000000.0000,2.0000,22000000.0000
50%,1984.0000,620.0000,34000000.0000,202106.0000,10000000.0000,4.0000,45000000.0000
75%,1992.0000,690.0000,48000000.0000,202205.0000,30000000.0000,6.0000,83000000.0000
max,2008.0000,1000.0000,10000000000.0000,20220630.0000,10000000000.0000,278.0000,7512000000.0000


**birth_year**
* 결측값이 존재함
* 신청자 중 가장 나이가 많은 사람은 1927년생이고, 가장 나이가 어린 사람은 2008년생이다.  
* 신청자들의 평균 나이는 1983년생으로 보인다.

**credit_score**  
신용 점수는 0점 ~ 1000점 만점을 기준으로 한다.
* 결측값이 존재함
* 가장 신용 점수가 낮은 점수는 60점이고, 가장 높은 점수는 1000점이다.

**yearly_income**
* 결측값이 존재함
* 평균 연봉은 약 4000만원대이며, 가장 낮은 연봉은 0이고 가장 높은 연봉은 100억이다.
* 50%, 75% 위치 값으로 보아 right skewed 되어있을 것이다.
* outlier 처리가 필요할 수도 있음.

**company_enter_month**
* 결측값이 존재함
* 평균 입사 연도는 float 값으로 처리되어 추후 연도-월로 변환이 필요하다.

**desired_amount**
* 결측값이 존재함
* 가장 낮은 대출 희망 금액은 0원이며, 가장 높은 대출 희망 금액은 100억이다.
* 평균 대출 희망 금액은 3000만원대이며, 중위값보다 큰 것으로 보아 right skewed 되어있을 것이다.
* outlier 처리가 필요할 수도 있음.

**existing_loan_cnt**
* 결측값이 존재함
* 가장 적은 기대출수는 1건이고, 가장 많은 기대출수는 278건이다.
* 평균 기대출수는 약 4~5건이다.
* outlier 처리가 필요할 수도 있음.

**existing_loan_amount**
* 결측값이 존재함
* 가장 적은 기대출금액은 0원이고, 가장 많은 기대출금액은 75억이다.
* 평균 기대출금액은 7000만원 정도이며, 중위값보다 큰 것으로 보아 right skewed 되어있을 것이다.

**종합해서**, interval 변수들의 범위가 모두 다르기 때문에 스케일링을 해주는 것이 좋을 것이며,  
분포 확인 및 outlier 확인이 필요해보인다.


### Binary variables

In [13]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
user_spec[v].describe()

,gender,personal_rehabilitation_yn,personal_rehabilitation_complete_yn
count,1381255.0000,806755.0000,190862.0000
mean,0.6764,0.0158,0.0071
std,0.4678,0.1245,0.0839
min,0.0000,0.0000,0.0000
25%,0.0000,0.0000,0.0000
50%,1.0000,0.0000,0.0000
75%,1.0000,0.0000,0.0000
max,1.0000,1.0000,1.0000


**gender**
* 성별의 평균값은 "1"의 비율로 해석할 수도 있다. 따라서 "1"이 NaN값이 아닌 전체 성별의 67%정도를 차지하는 것으로 해석할 수 있다.
* 결측값이 존재한다.

**personal_rehabilitation_yn**
* 성별과 마찬가지로 개인회생자의 비율이 NaN값이 아닌 전체 개인회생자 수의 1%정도를 차지하는 것으로 해석할 수 있다.
* 결측값이 존재한다.

**personal_rehabilitation_complete_yn**
* 성별과 마찬가지로 개인회생자 납입 완료 비율이 NaN값이 아닌 전체 개인회생자 납입 여부 수의 0.7% 정도를 차지하는 것으로 해석할 수 있다. 
* 결측값이 존재한다.

**종합해서**, binary 변수들은 gender를 제외하면 모두 **strongly imbalanced** 하다.

### Categorical variables의 카디널리티 확인  
  
Cardinality는 데이터의 unique한 값이라 할 수 있다.

In [48]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    uniq_values = user_spec[f].value_counts().shape[0]
    print(f'Variable {f} - {uniq_values} unique values')

Variable income_type - 6 unique values
Variable employment_type - 4 unique values
Variable houseown_type - 4 unique values
Variable purpose - 16 unique values


* Purpose 피처만 높은 카디널리티를 보인다.

### 결측값 확인

In [15]:
user_spec.isnull().sum() / user_spec.shape[0] * 100 # 퍼센트 단위로 만들어 줌

application_id                         0.0000
user_id                                0.0000
birth_year                             0.9296
gender                                 0.9296
insert_time                            0.0000
credit_score                           7.5394
yearly_income                          0.0065
income_type                            0.0061
company_enter_month                   12.3195
employment_type                        0.0061
houseown_type                          0.0061
desired_amount                         0.0061
purpose                                0.0061
personal_rehabilitation_yn            42.1356
personal_rehabilitation_complete_yn   86.3104
existing_loan_cnt                     14.2414
existing_loan_amt                     22.5054
dtype: float64

* 개인회생자 변수들이 모두 가장 높은 결측값을 가진다.
* categorical 변수들에도 결측값이 존재하며, 이것들은 -1로 둘 수도 있다.
* 결측값들을 대치시키는 것도 좋지만, 이 데이터에서는 다른 의미를 가질 수도 있기 때문에 확인해볼 필요가 있다.